# Neural Style Transfer

Nueral Style Transfer using PyTorch. Original paper by *Leon A. Gatys*, *Alexander S. Ecker* and *Matthias Bethge* [here](https://arxiv.org/abs/1508.06576). Majority of the code used for this repository is authored by *Alexis Jacq* and edited by *Winston Herring*. The link for their article can be found [here](https://pytorch.org/tutorials/advanced/neural_style_tutorial.html).

### Import packages

In [1]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F

# efficient gradient descents
import torch.optim as optim 

from PIL import Image
import matplotlib.pyplot as plt

# transform PIL images into tensors
import torchvision.transforms as transforms 

import torchvision.models as models

#to deep copy the models
import copy

If we have a GPU available, use it! If not, we use the CPU which will take a little longer to run the network.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### What are we trying to accomplish?

To keep it simple, **Neural Transfer** involves reproducing the contents of an *input image* with the artistic style of a *style image*. For example, can we take the *style* of the watercolor painting below and the *content* of the turtle image in order to produce a third image which is a 'combination' of the two. As you can see, this can lead to very fun results!

**Source:** Pytorch Advanced Tutorials
![Example](https://pytorch.org/tutorials/_images/neuralstyle.png)

### Basic Theory 

Two distances, one for the content (D<sub>C</sub>) and one for the style (D<sub>S</sub>) are defined. They measure how different two images are content-wise and stylistically. 

A third image, called the input image, which can be white noise or the content image itself is then transformed so that its content-distance with the content-image and its style-distance with the style-image are both minimized.

### Load the content and style images

The original PIL images have values between 0 and 255. These need to be converted to torch tensors, with values between 0 and 1. This is becasue neural networks from the torch library are trained on the same range. Passing inputs in the 0 to 255 range will render the activated feature maps of the network useless. Another important detail is that the *content* and *style* images need to have the same dimensions. We'll resize them to a fixed value to ensure this.

In [3]:
# use smaller image size if gpu isn't available
if torch.cuda.is_available():
    imsize = (512, 512) 
else:
    imsize = (256, 256)  

# Resize image and transform to torch tensor
tfms = [
    transforms.Resize(imsize),
    transforms.ToTensor()
]
loader = transforms.Compose(tfms)

def image_loader(img_path):
    img = Image.open(img_path)
    # Insert 1 in shape of the tensor at axis 0
    # Extra dimension is required to fit the network's input dimensions
    img = loader(img).unsqueeze(0)
    return img.to(device, torch.float)


style_img = image_loader("./data/dali.jpg")
content_img = image_loader("./data/sarthak.jpg")

assert style_img.size() == content_img.size(), \
    "we need to import style and content images of the same size"